# ML Pipeline Preparation
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [28]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib as plt
from sqlalchemy import create_engine
import re
import nltk
nltk.download(['punkt','stopwords','wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /Users/jaywang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaywang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jaywang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jaywang/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [18]:
# load data from database
engine = create_engine('sqlite:///Disaster_response_data.db')
df = pd.read_sql('SELECT * FROM Disaster_data', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
X = df['message']
Y = df.iloc[:,4:]

In [20]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [21]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [22]:
def tokenize(text):
    # Normalize
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    # Tokenization
    words = word_tokenize(text)
    # Remove stop words
    words = [w for w in words if w not in stopwords.words('english')]
    # Lemmatize
    lemmatizer = [WordNetLemmatizer().lemmatize(w) for w in words]
    return lemmatizer

tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [23]:
for message in X:
    tokens = tokenize(message)
    print(message)
    print(tokens,'\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 

Information about the National Palace-
['information', 'national', 'palace'] 

Storm at sacred heart of jesus
['storm', 'sacred', 'heart', 'jesus'] 

Please, we need tents and water. We are in Silo, Thank you!
['please', 'need', 'tent', 'water', 'silo', 'thank'] 

I would like to receive the messages, thank you
['would', 'like', 'receive', 'message', 'thank'] 

I am 

How can we find help and food in fontamara 43 rue menos
['find', 'help', 'food', 'fontamara', '43', 'rue', 'menos'] 

SOS SOS, please provide police officers on the streets as they are very insecure
['so', 'so', 'please', 'provide', 'police', 'officer', 'street', 'insecure'] 

.. .. . we are here our mother has no more water. if you are there let us know ( location not provided )
['mother', 'water', 'let', 'u', 'know', 'location', 'provided'] 

People needing water with baby Location : Beach 24
['people', 'needing', 'water', 'baby', 'location', 'beach', '24'] 

help is needed in fond parisien, no idea where that is, but any haitian can tell you
['help', 'needed', 'fond', 'parisien', 'idea', 'haitian', 'tell'] 

We're asking for water, medical supply, food
['asking', 'water', 'medical', 'supply', 'food'] 

My home is at Gressier, it has almost collapsed. I would like to distroy it before it collapses completly.
['home', 'gressier', 'almost', 'collapsed', 'would', 'like', 'distroy', 'col

We are in dire need of food, water and temporary shelter. We need to know where we can get supplies. We are located in Frere Boukan, La plaine.
['dire', 'need', 'food', 'water', 'temporary', 'shelter', 'need', 'know', 'get', 'supply', 'located', 'frere', 'boukan', 'la', 'plaine'] 

The road is damaged and there is no gas, would like to find a motocycle. It costs 500 to go to Port-au-Prince. No one has any money.
['road', 'damaged', 'gas', 'would', 'like', 'find', 'motocycle', 'cost', '500', 'go', 'port', 'au', 'prince', 'one', 'money'] 

I woul like to know if aide is only available in pap as the provinces where badly hit as well
['woul', 'like', 'know', 'aide', 'available', 'pap', 'province', 'badly', 'hit', 'well'] 

We have a factory that is on FIRE on road to the airport near Sogebank. It's starting to burn several nearby houses with documents left in them. Please come and help us!!!
['factory', 'fire', 'road', 'airport', 'near', 'sogebank', 'starting', 'burn', 'several', 'nearby',

Can someone who has a visa travel out. What can he do?
['someone', 'visa', 'travel'] 

there are alot of victimes that have arrived ath the Marchand Desalin and nan Ranboto hospital. they do not have enough supplys or specialist ( surgeons ) i would like this looked into
['alot', 'victimes', 'arrived', 'ath', 'marchand', 'desalin', 'nan', 'ranboto', 'hospital', 'enough', 'supply', 'specialist', 'surgeon', 'would', 'like', 'looked'] 

help us at delmas 65 orchidee street, we are thirsty. thanks
['help', 'u', 'delmas', '65', 'orchidee', 'street', 'thirsty', 'thanks'] 

we want people to know that people that were in Port au Prince that came to Cap-Haitian have no food to eat. ( Cap-Haitien is in the north, 250 km from Port au Prince )
['want', 'people', 'know', 'people', 'port', 'au', 'prince', 'came', 'cap', 'haitian', 'food', 'eat', 'cap', 'haitien', 'north', '250', 'km', 'port', 'au', 'prince'] 

And in St. Marc, just came to save their lives
['st', 'marc', 'came', 'save', 'life'] 

t

Please, what can you do for us. We do not have anything.
['please', 'u', 'anything'] 

Cannot locate on map, not sure if in Port-au-Prince We need aid nan delma (?) 9 rue de cosquer 16
['locate', 'map', 'sure', 'port', 'au', 'prince', 'need', 'aid', 'nan', 'delma', '9', 'rue', 'de', 'cosquer', '16'] 

We heard that they are giving things out. We didn't get anything to eat or drink. Please bring some aide to carrefour!
['heard', 'giving', 'thing', 'get', 'anything', 'eat', 'drink', 'please', 'bring', 'aide', 'carrefour'] 

My house is destroyed in Carrefour. My house is destroyed in Port-au-Prince. The money that the government gave me is lost. My children die, but I am not hurt. Please if you can help me protect the house. ...
['house', 'destroyed', 'carrefour', 'house', 'destroyed', 'port', 'au', 'prince', 'money', 'government', 'gave', 'lost', 'child', 'die', 'hurt', 'please', 'help', 'protect', 'house'] 

(missing letter on first. probably t) My entire family would like you all to (

...

#sandyhatesbooks, but #jacklovesbooks. Come buy a book and support @powerhousearena! (@Powerhouse Arena) http://t.co/x9bQJSAYI_BX shirt, @nycservice and @friendsofvcp after volunteering to clean up Van Cortlandt Park. #sandybronx #volunteer
['sandyhatesbooks', 'jacklovesbooks', 'come', 'buy', 'book', 'support', 'powerhousearena', 'powerhouse', 'arena', 'http', 'co', 'x9bqjsayi', 'bx', 'shirt', 'nycservice', 'friendsofvcp', 'volunteering', 'clean', 'van', 'cortlandt', 'park', 'sandybronx', 'volunteer'] 

Volunteering! Hot food 4 those w/o power, shelter, or otherwise been affected by Sandy! #disasterrelief #service #nj http://t.co/71MZ85NA
['volunteering', 'hot', 'food', '4', 'w', 'power', 'shelter', 'otherwise', 'affected', 'sandy', 'disasterrelief', 'service', 'nj', 'http', 'co', '71mz85na'] 

Way off my twitter game sandy is really messing with me
['way', 'twitter', 'game', 'sandy', 'really', 'messing'] 

This was long Eid I mean hurricane break
['long', 'eid', 'mean', 'hurricane', '

Just saw pictures of my breezy house and lost it #fuckyousandy #wantlifebacm
['saw', 'picture', 'breezy', 'house', 'lost', 'fuckyousandy', 'wantlifebacm'] 

Best #businessweek cover. Ever. #globalwarming #sandy #lovenyc @Tbwa Worldwide (5th Floor) http://t.co/7rGy4oHI@HiddenHoboken spread the word that the Malibu diner is up and running! #hobokensandy #hobokenpowerThis snow storm better be sexy.I'm so burnt out and exhausted from work as a result of this hurricane Sandy.Were out in Staten island, and back in far Rockaway tomorrow. #TLCSandyrelief If you can help 1633 Centre st. @ChristTab @thelegacycenter
['best', 'businessweek', 'cover', 'ever', 'globalwarming', 'sandy', 'lovenyc', 'tbwa', 'worldwide', '5th', 'floor', 'http', 'co', '7rgy4ohi', 'hiddenhoboken', 'spread', 'word', 'malibu', 'diner', 'running', 'hobokensandy', 'hobokenpowerthis', 'snow', 'storm', 'better', 'sexy', 'burnt', 'exhausted', 'work', 'result', 'hurricane', 'sandy', 'staten', 'island', 'back', 'far', 'rockaway', 

Full moon #fullmoonnyc #aftersandy's @West Village http://t.co/5qi9SsKj
['full', 'moon', 'fullmoonnyc', 'aftersandy', 'west', 'village', 'http', 'co', '5qi9sskj'] 

Blessed and highly favored!! Im gonna make the best out today!! Ready for this snow storm!! Wanna be cuddled up with him when it come later!
['blessed', 'highly', 'favored', 'im', 'gon', 'na', 'make', 'best', 'today', 'ready', 'snow', 'storm', 'wan', 'na', 'cuddled', 'come', 'later'] 

ealDonaldTrump #sandy. Class act Mr. Trump. Keep posting. Your the true "real" source of information.Snow/sleet/rain in NYC. Usually I love this, but #toosoon #SandyAftermath
['ealdonaldtrump', 'sandy', 'class', 'act', 'mr', 'trump', 'keep', 'posting', 'true', 'real', 'source', 'information', 'snow', 'sleet', 'rain', 'nyc', 'usually', 'love', 'toosoon', 'sandyaftermath'] 

JFK -&gt; SFO Trying to get out before the storm. First time on @VirginAmerica. (@Virgin America Terminal) http://t.co/lXBwcH17
['jfk', 'gt', 'sfo', 'trying', 'get', 'storm

Gas rationing finally hits NY. I haven't been able to get gas since before sandy. Thank God for public transportation!@jaac_nic I don't like how this storm kept us apart _The east coast is really in need of help after the storm (Sandy)..please donate in whatever little way you can.Having a blast at @LoxPopuli #SandyRelief fundraiser. Met @jensnow and @NRussDaughters. Delicious food, great booze. @DanielSquadron spoke!Still drankin in support of #Sandy relief w/ @queensmuseum #williamsburgdinein @mloezartz @duchessdw @alexjsimons http://t.co/XfsYTEAb
['gas', 'rationing', 'finally', 'hit', 'ny', 'able', 'get', 'gas', 'since', 'sandy', 'thank', 'god', 'public', 'transportation', 'jaac', 'nic', 'like', 'storm', 'kept', 'u', 'apart', 'east', 'coast', 'really', 'need', 'help', 'storm', 'sandy', 'please', 'donate', 'whatever', 'little', 'way', 'blast', 'loxpopuli', 'sandyrelief', 'fundraiser', 'met', 'jensnow', 'nrussdaughters', 'delicious', 'food', 'great', 'booze', 'danielsquadron', 'spoke'

KeyboardInterrupt: 

In [26]:
def display_results(y_test, y_pred):
    """[summary]
    
    Arguments:
        y_test {[type]} -- [description]
        y_pred {[type]} -- [description]
    """
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_tran, X_test, y_train, y_test = train_test_split(X, y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.